In [ ]:
Prerequisites:
1. pip install helium
2. Install chromedriver.exe


In [ ]:
Challenges:
1. Didnot implement scroll to the end logic as there are large number of details in the page
2. Used helium instead as selenium

In [ ]:
import csv
from helium import *
from bs4 import BeautifulSoup as bs
import logging

logging.basicConfig(filename="course_logging.log", level=logging.DEBUG, format='%(asctime)s %(levelname)s %(message)s')


class retrieve_course_content:

    def get_course_info(self):

        write_recs_list = []

        try:
            # specify the url
            url = 'https://courses.ineuron.ai'
            browser = start_chrome(url, headless=True)
            logging.info('Url opened successfully')
        except Exception as e:
            logging.error('Error occurred while opening the url')
            logging.exception('Exception while opening the url' + str(e))

        try:
            # Find all courses in the courses.ineuron.ai page - Did not implement scroll to the end logic
            all_courses = find_all(S('.Course_course-title__2rA2S'))
            course_name = [item.web_element.text for item in all_courses]

            # Get the respective course urls from the href tags
            course_urls = []
            soup = bs(browser.page_source, 'html.parser')
            courses = soup.find_all('div', {'class': 'Course_right-area__1XUfi'})
            for item in courses:
                for i in item.find_all('a', href=True):
                    course_urls.append(url + i['href'])
            course_urls = list(set(course_urls))  # The href returns duplicate urls, so eliminate duplicates
            logging.info('Course name and course links retrieved successfully')

            for name in course_name:
                # matching course link from the list of urls for the respective course name
                link = [s for s in course_urls if name.split()[0] in s]
                link = (" ".join(link))  # converting the list to a string

                # Open the link of every course
                course_link = start_chrome(link, headless=True)
                course_content = bs(course_link.page_source, 'html.parser')

                # find the course description tag
                course_desc = course_content.find('div', {'class': 'Hero_course-desc__26_LL'}).text

                # find what you'll learn tag and retrieve all the pointers under "What you'll learn" subheading
                course_learning = course_content.find('div', {'class': 'CourseLearning_card__WxYAo card'})

                course_topics = course_learning.find_all('li')  # returns the list of all 'li' tags

                topics_covered = []

                for topic in course_topics:
                    topic_html_tag = str(topic)
                    end_pos = len(topic_html_tag) - 5
                    topics_covered.append(topic_html_tag[4:end_pos])

                logging.info('Course description and course topics retrieved successfully')

                # Retrieve all the pointers under Requirements subheading
                course_reqs = course_content.find('div', {'class': 'CourseRequirement_card__3g7zR requirements card'})

                # Retrieve all the pointers under Requirements subheading
                reqs_tags = course_reqs.find_all('li')  # returns the list of all 'li' tags

                reqs_needed = []

                for req in reqs_tags:
                    req_html_tag = str(req)
                    end_pos = len(req_html_tag) - 5
                    reqs_needed.append(req_html_tag[4:end_pos])

                logging.info('Course requirements retrieved successfully')

                write_dict = {'Course Name': name, 'Course Link': link, 'Course Description': course_desc,
                          'Topics Covered': topics_covered, 'Course Requirements': reqs_needed}
                write_recs_list.append(write_dict)

                logging.info('Writing retrieved contents in a dictionary')
                self.write_csv_file(write_recs_list)
        except:
            logging.error('Error occurred while parsing the html tags')
            logging.exception('Exception occurred while parsing the html tags' + str(e))

    def write_csv_file(self, write_recs_list):
        try:
            with open('Ineuron_Courses.csv', 'w', newline='') as csvfile:
                fieldnames = ['Course Name', 'Course Link', 'Course Description', 'Topics Covered',
                          'Course Requirements']
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writeheader()
                for rec in write_recs_list:
                    writer.writerow(rec)
                logging.info('Successful completion of writing records in csv file')
        except:
            logging.error('Error occurred while writing records in csv file')
            logging.exception('Exception occurred while writing records in csv file' + str(e))

data_scraping = retrieve_course_content()
data_scraping.get_course_info()

